# cuSpatial API demo
GTC April 2023 Michael Wang and Thomson Comer


The following notebook demonstrates the use of cuSpatial to perform analytics using large datasets.

The structure of the notebook is as follows:
1. Imports
2. Read datasets: National Address Database (NAD), NYC Taxi Boroughs Polygons, 2015 NYC Taxi pickup/dropoff information with lon/lat. Also convert epsg:2263 (NYC Long Island) to WSG.
3. Convert separate lon/lat columns in DataFrames into cuspatial.GeoSeries
4. Count the number of pickups and dropoffs per zone
5. Compute street names for each pickup and dropoff
6. Calculate the number of addresses per zone

In [1]:
import cudf
import cuspatial
import geopandas
import cupy as cp
import pandas as pd

In [2]:
# I/O (18GB NAD, 265 borough polygons, 7m taxi pickups and 16m taxi dropoffs.
NAD_Street = cudf.read_csv('NAD_r11.txt', usecols=[
    'State',
    'StN_PreDir',
    'StreetName',
    'StN_PosTyp',
    'Add_Number'
])
NAD = cudf.read_csv('NAD_r11.txt', usecols=[
    'State',
    'Longitude',
    'Latitude',
])
NAD = NAD[NAD['State'] == 'NY']
NAD_Street = NAD_Street[NAD_Street['State'] == 'NY']
# Read taxi_zones.zip shapefile with GeoPandas, then convert to epsg:4326 for lon/lat
host_zones = geopandas.read_file('taxi_zones.zip')
host_lonlat = host_zones.to_crs(epsg=4326)
zones = cuspatial.from_geopandas(host_lonlat)
taxi2015 = cudf.read_csv('taxi2015.csv')

In [3]:
# Utility function to convert dataframes into GeoSeries

def make_geoseries_from_lon_lat(lon, lat):
    # Scatter the two columns into one column
    assert len(lon) == len(lat)
    xy = cudf.Series(cp.zeros(len(lon) * 2))
    xy[::2] = lon
    xy[1::2] = lat

    return cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(xy._column))


In [4]:
# Convert DataFrames to GeoSeries

pickups = make_geoseries_from_lon_lat(
    taxi2015['pickup_longitude'],
    taxi2015['pickup_latitude']
)
addresses = make_geoseries_from_lon_lat(
    NAD['Longitude'],
    NAD['Latitude']
)

In [5]:
# Count the number of dropoffs and pickups per zone, one at a time.

pickup_counts = zones['geometry'].contains_properly(pickups, align=False).sum()
zones['pickup_counts'] = pickup_counts

AttributeError: 'NumericalColumn' object has no attribute 'leaves'

In [16]:
def quadtree(polygons, points):
    poly_points_x = polygons.polygons.x
    poly_points_y = polygons.polygons.y
    poly_offsets = polygons.polygons.part_offset
    poly_ring_offsets = polygons.polygons.ring_offset
    test_points_x = points.points.x
    test_points_y = points.points.y
    scale = 50
    max_depth = 7
    min_size = 125
    x_max = poly_points_x.max()
    x_min = poly_points_x.min()
    y_max = poly_points_y.max()
    y_min = poly_points_y.min()
    point_indices, quadtree = cuspatial.quadtree_on_points(
        test_points_x,
        test_points_y,
        x_min,
        x_max,
        y_min,
        y_max,
        scale,
        max_depth,
        min_size,
    )
    poly_bboxes = cuspatial.polygon_bounding_boxes(
        poly_offsets, poly_ring_offsets, poly_points_x, poly_points_y
    )
    intersections = cuspatial.join_quadtree_and_bounding_boxes(
        quadtree, poly_bboxes, x_min, x_max, y_min, y_max, scale, max_depth
    )
    polygons_and_points = cuspatial.quadtree_point_in_polygon(
        intersections,
        quadtree,
        point_indices,
        test_points_x,
        test_points_y,
        poly_offsets,
        poly_ring_offsets,
        poly_points_x,
        poly_points_y,
    )
    polygons_and_points['point_index'] = point_indices.iloc[
        polygons_and_points['point_index']
    ].reset_index(drop=True)
    return polygons_and_points

In [17]:
addresses_pip = quadtree(zones['geometry'], addresses)
addresses_pip

,polygon_index,point_index
0,1,5648100
1,1,5648101
2,34,5202801
3,34,5202802
4,34,5202803
...,...,...
966784,353,5368821
966785,353,5368822
966786,353,5368823
966787,353,5368824


In [18]:
pickups_pip = quadtree(zones['geometry'].iloc[0:120], pickups)
pickups_pip

,polygon_index,point_index
0,0,44084
1,0,76169
2,0,129737
3,0,177939
4,0,219859
...,...,...
3408316,167,12253904
3408317,167,12574064
3408318,167,12634955
3408319,167,12666699


In [19]:
# a mapping from parts to polygons

def pip_result_to_id_map(polygons, pip_result):
    offsets = cp.array(polygons.polygons.geometry_offset)
    polygon_lengths = offsets[1:] - offsets[:-1]
    parts = polygons.polygons.part_offset
    polygon_map = cp.arange(len(polygon_lengths)).repeat(polygon_lengths.tolist())
    idx_df = cudf.DataFrame({
        'OBJECTID': polygon_map,
        'polygon_index': cp.arange(len(parts)-1)
    })
    return pip_result.merge(idx_df, on="polygon_index").drop('polygon_index', axis=1)
borough_addresses = pip_result_to_id_map(zones['geometry'], addresses_pip)
borough_pickups = pip_result_to_id_map(zones['geometry'], pickups_pip)
borough_addresses

,point_index,OBJECTID
0,5209367,2
1,5209368,2
2,5209369,2
3,5209370,2
4,5209371,2
...,...,...
966784,5368821,262
966785,5368822,262
966786,5368823,262
966787,5368824,262


In [ ]:
# Let's compute the practical limit for actual boroughs.

pickup_counts = pickup_counts.fillna(0)
address_counts = address_counts.fillna(0)
comparison_size = pickup_counts.sort_index()['polygon_index'] * address_counts.sort_index()['polygon_index']
comparison_size.fillna(0).unique()

In [ ]:
borough_addresses

In [ ]:
# Let's make two GeoSeries: For each borough, create a GeoSeries with all address Points
# repeated the number of times there are pickups in that borough, and another GeoSeries with
# the opposite: all pickups Points repeated the number of times there are addresses in that
# borough.

borough_address_point_ids = borough_addresses['point_index'][borough_addresses['OBJECTID'] == 2]
pickups_count = len(borough_pickups[borough_pickups['OBJECTID'] == 2])
addresses_dataframe = NAD.iloc[
    borough_address_point_ids
].tile(pickups_count)
addresses_ids = borough_address_point_ids.tile(pickups_count).reset_index(drop=True)
borough_pickup_point_ids = borough_pickups['point_index'][borough_pickups['OBJECTID'] == 2]
addresses_count = len(borough_addresses[borough_addresses['OBJECTID'] == 2])
pickups_dataframe = taxi2015[[
    'pickup_longitude',
    'pickup_latitude'
]].iloc[borough_pickup_point_ids].tile(addresses_count)
pickups_ids = borough_pickup_point_ids.tile(addresses_count).reset_index(drop=True)
pickup_points = make_geoseries_from_lon_lat(
    pickups_dataframe['pickup_longitude'],
    pickups_dataframe['pickup_latitude']
)
address_points = make_geoseries_from_lon_lat(
    addresses_dataframe['Longitude'],
    addresses_dataframe['Latitude']
)

In [ ]:
# get the list of addresses and their indices that are closest to a pickup point

distances = cuspatial.pairwise_point_distance(pickup_points, address_points)
pickups_indices = cp.arange((borough_pickups['OBJECTID'] == 2).sum())
pickups_index_map = pickups_indices.repeat((borough_addresses['OBJECTID'] ==2).sum())
distances_df = cudf.DataFrame({'distances': distances, 'index': pickups_index_map})
nearest_pickups = pickup_points.iloc[distances_df.groupby('index').idxmin()['distances']]
NAD_addresses_ids = addresses_ids[nearest_pickups.index]
taxi2015_pickups_ids = pickups_ids[nearest_pickups.index]

In [ ]:
# concatenate address fields

blanks = cudf.Series([' '] * len(NAD_Street))
blanks.index = NAD_Street.index
NAD_Street['StN_PreDir'] = NAD_Street['StN_PreDir'].fillna('')
street_names = NAD_Street['Add_Number'].astype('str').str.cat(
    NAD_Street['StN_PreDir']
).str.cat(
    blanks
).str.cat(
    NAD_Street['StreetName']
).str.cat(
    blanks
).str.cat(
    NAD_Street['StN_PosTyp']
)
street_names

In [ ]:
# The number of addresses per borough

address_counts = borough_addresses.groupby('OBJECTID').count()
count_df = cudf.DataFrame({
    'OBJECTID': address_counts.index,
    "Address Count": address_counts.polygon_index
})

In [ ]:
# Add the address counts back to the zones dataframe

# Cudf doesn't know how to print `geometry` columns, so put it back into cuspatial
merged_zones = cuspatial.GeoDataFrame(zones.merge(count_df))
merged_zones.head()

In [ ]:
# We have the street names in NAD above, let's add the street name to each pickup

borough_addresses
print(NAD.head())
print(taxi2015.head())
print(pickups.head())